In [25]:
## Import relevant modules

from numpy import*
from matplotlib.pyplot import*
from iris import*
from iris.analysis import*
import iris.quickplot as quickplot
from iris.coord_categorisation import*
from scipy.stats import*
from scipy.stats.mstats import*
import netCDF4 as nc
import numpy.ma as ma

In [33]:
miroc6_historical = iris.load_cube('/home/cj362/ENSO_data/tos/miroc6_historical/tos_Omon_MIROC6_historical_r1i1p1f1_gn_195001-201412_regridded.nc')
#print(miroc6_historical)

# Define Nino3.4 region
north=5
east=-120
south=-5
west=-170

# Subset files to Nino3.4 region
miroc6_historical_lon = miroc6_historical.intersection(longitude =(west,east))
miroc6_historical_nino34=miroc6_historical_lon.intersection(latitude=(south,north))

## Monthly mean SST across Nino3.4 region 2015-2100 (1032 months = 1032 output values)
miroc6_historical_nino34.coord('longitude').guess_bounds() # Define longitude bounds
miroc6_historical_nino34.coord('latitude').guess_bounds() # Define latitude bounds
miroc6_historical_nino34_mon_mean_grid_area = iris.analysis.cartography.area_weights(miroc6_historical_nino34) # Create grid area
miroc6_historical_nino34_mon_mean = miroc6_historical_nino34.collapsed(['longitude', 'latitude'],\
                              iris.analysis.MEAN, weights=miroc6_historical_nino34_mon_mean_grid_area) # Collape values

# Printing data from the extracted nino3.4 region 
miroc6_historical_nino34_mon_mean_data = miroc6_historical_nino34_mon_mean.data # Extract collapsed file data
print("miroc6_historical_nino3.4 region monthly mean SST data:")
print
print(miroc6_historical_nino34_mon_mean_data)
print


######################## Using Paul's code he sent through to extract monthly values between 2000 and 2014: ###########################
# cube names in your supplied code have been adjusted to the file that is used in this cell. This file began as "mircoc6_historical" (at top)...
# ...but has now been adjusted just to account for the data in the nino3.4 region hence file is called "miroc6_historical_nino34_mon_mean" (file has been printed just below)

print("miroc6_historical_nino3.4 region_monthly mean file:")
print
print(miroc6_historical_nino34_mon_mean)

# Specify years you want (adjusted from code you supplied to appropriate years)
start_year = 2000
end_year = 2014

# Find which items in this cube that relate to these years
iris.coord_categorisation.add_year(miroc6_historical_nino34_mon_mean, 'time', name='year') 
loc1 = np.where((miroc6_historical_nino34_mon_mean.coord('year').points >= start_year) & (miroc6_historical_nino34_mon_mean.coord('year').points <= end_year))

# Extract those years into a new variable called cube1
cube1 = miroc6_historical_nino34_mon_mean[loc1]

# Printing all objects your code has supplied
print(loc1)
print("#################################################################################################")
print("#################################################################################################")
print("#################################################################################################")
print
print("As you can see it just prints the month value numbers, not the SST values, and more so the incorrect month value entries (600-779) instead of 601 to 780")
print
print
print(cube1)
print("#################################################################################################")
print("#################################################################################################")
print("#################################################################################################")
print
print
print
print
print
print

#########################End of using Paul's code #################################################

# As you can see it extracts almost correctly, however it extracts the values one value less (i.e. 600-779) when I need values "601-780"

################# This is what I was experimenting with to see if I could extract it specifically on a monthly basis rather than through the yearly values: ################

print("My method result:")
# Subset files for months between 2000-2014 (180 entries including #601 and #780)
start_month = 601
end_month = 780

iris.coord_categorisation.add_month(miroc6_historical_nino34_mon_mean, 'time', name='month')
loc2 = np.where((miroc6_historical_nino34_mon_mean.coord('month').points >= start_month) & (miroc6_historical_nino34_mon_mean.coord('month').points <= end_month))

# Extract those months into a new variable called cube2
cube2 = miroc6_historical_nino34_mon_mean[loc2]

# Print objects from your code but with my adjustment 
print(loc2)
print("#################################################################################################")
print("#################################################################################################")
print("#################################################################################################")
print
print
print(cube2)
print("#################################################################################################")
print("#################################################################################################")
print("#################################################################################################")
print

## Save monthly mean SST array as .txt file source: https://www.kite.com/python/examples/5522/numpy-save-an-array-to-a-text-file
# Format: np.savetxt("outputfilename",np.specified array you want to save)
np.savetxt("miroc6_historical_nino34_mon_mean_data.txt",np.array(miroc6_historical_nino34_mon_mean_data))


miroc6_historical_nino3.4 region monthly mean SST data:

[25.32474632 25.83310178 26.34852826 27.1255497  27.37143072 27.17676263
 26.52116229 25.62509165 25.3603995  25.05208333 25.18982756 25.38753286
 25.48424766 25.70427341 26.3992915  27.1944176  27.25774265 27.29479589
 26.82446643 26.26107446 25.56397257 25.41505469 25.1606249  25.26574653
 25.46041243 25.86290067 26.54714622 27.57624874 27.27601188 26.95565086
 26.46940784 25.10334477 24.62435845 24.53433312 24.00797303 24.1733722
 24.72792884 25.0602154  25.34307555 25.82217694 26.05153954 25.91473698
 25.14488152 24.22615352 23.82840021 23.82779471 23.98282411 24.07819172
 24.60148363 24.99411322 25.78430948 26.18685593 26.5774529  26.66139389
 26.44600446 26.1139746  25.89489605 25.84213937 25.50886897 25.40495981
 25.48129229 25.82247646 26.50822228 27.12881471 27.65796463 27.59646528
 27.49738157 27.17761391 26.95484625 26.95311697 26.7043212  26.63522163
 26.54149415 26.65425062 27.13891936 27.52175157 28.03831101 28.0467

/home/cj362/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:79: DeprecationWarning: unorderable dtypes; returning scalar but in the future this will be an error


IndexError: Cannot index with zero length slice.

In [20]:
# Load in climate netCDF files

miroc6_ssp2 = iris.load_cube('/home/cj362/ENSO_data/tos/miroc6_ssp2_45/tos_Omon_MIROC6_ssp245_r1i1p1f1_gn_201501-210012_reggridded.nc')

# print(miroc6_ssp2)

# Define Nino3.4 region
north=5
east=-120
south=-5
west=-170

# Subset files to Nino3.4 region
miroc6_ssp2_lon=miroc6_ssp2.intersection(longitude =(west,east))
miroc6_ssp2_nino34=miroc6_ssp2_lon.intersection(latitude=(south,north))


##  Mean SST for each grid cell between 2015-2100 (714 grid squares = 714 output values)
# miroc6_ssp2_nino34_gridsquare_mean = miroc6_ssp2_nino34.collapsed('time', iris.analysis.MEAN) 
# print(miroc6_ssp2_nino34_gridsquare_mean)
# miroc6_ssp2_nino34_mgridsquare_mean_data = miroc6_ssp2_nino34_gridsquare_mean.data
# print(miroc6_ssp2_nino34_gridsquare_mean_data)

## Monthly mean SST across Nino3.4 region 2015-2100 (1032 months = 1032 output values)
miroc6_ssp2_nino34.coord('longitude').guess_bounds() # Define longitude bounds
miroc6_ssp2_nino34.coord('latitude').guess_bounds() # Define latitude bounds
miroc6_ssp2_nino34_mon_mean_grid_area = iris.analysis.cartography.area_weights(miroc6_ssp2_nino34) # Create grid area
miroc6_ssp2_nino34_mon_mean = miroc6_ssp2_nino34.collapsed(['longitude', 'latitude'],\
                              iris.analysis.MEAN, weights=miroc6_ssp2_nino34_mon_mean_grid_area) # Collape values

print(miroc6_ssp2_nino34_mon_mean) # Print collapsed file

miroc6_ssp2_nino34_mon_mean_data = miroc6_ssp2_nino34_mon_mean.data # Extract collapsed file data
print(miroc6_ssp2_nino34_mon_mean_data)
print
# print(type(miroc6_ssp2_nino34_mon_mean_data))



## Quick, non-truncated view of all entries within monthly mean SST array
# for i in range (0, len(miroc6_ssp2_nino34_mon_mean_data)):
    
    # print(miroc6_ssp2_nino34_mon_mean_data[i])
    

# View number of dimensions, shape, size of numpy source: https://note.nkmk.me/en/python-numpy-ndarray-ndim-shape-size/
# array_name.ndim - dimensions
# array_name.shape - shape
# array_name.size - size (multiply sides of the shape)


## Save monthly mean SST array as .txt file source: https://www.kite.com/python/examples/5522/numpy-save-an-array-to-a-text-file
# Format: np.savetxt("outputfilename",np.specified array you want to save)
np.savetxt("miroc6_ssp2_nino34_mon_mean_data.txt",np.array(miroc6_ssp2_nino34_mon_mean_data))





## Identifying ENSO using ONI

# 

sea_surface_temperature / (degC)    (time: 1032)
     Dimension coordinates:
          time                           x
     Scalar coordinates:
          latitude: 0.0 degrees, bound=(-4.921875, 4.921875) degrees
          longitude: -145.0 degrees, bound=(-170.5, -119.5) degrees
     Attributes:
          CDI: Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
          CDO: Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/cdo)
          Conventions: CF-1.7 CMIP-6.2
          activity_id: ScenarioMIP
          branch_method: standard
          branch_time_in_child: 60265.0
          branch_time_in_parent: 60265.0
          cmor_version: 3.4.0
          comment: Temperature of upper boundary of the liquid ocean, including temperatures...
          creation_date: 2019-06-14T02:44:35Z
          data_specs_version: 01.00.29
          experiment: update of RCP4.5 based on SSP2
          experiment_id: ssp245
          external_variables: areacello
          forcing_ind